Importação da bibliotecas necessárias 

In [105]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
#from tqdm import tqdm
import asyncio
from tqdm.asyncio import tqdm 
import nest_asyncio
from time import sleep
import sqlite3

Download do Arquivo com a Lista da Empresa

In [48]:
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":"/Users/andrenunes/PycharmProjects/POS_PYTHON/downloads"}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(options=options)
driver.get("https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br")
btn_download = driver.find_element(By.PARTIAL_LINK_TEXT, "Download")
btn_download.click()
time.sleep(3)
driver.close()

In [76]:
def get_symbols():
    hoje = datetime.today()
    nome_arquivo = hoje.strftime('downloads/IBOVDia_%d-%m-%y.csv')
    with open(nome_arquivo, 'r', encoding='iso-8859-1') as arquivo:
        linhas = arquivo.readlines()
        nomes_alterados = [linha.split(';')[0] + ".SA" for linha in linhas[2:-2]]
    return nomes_alterados
symbols = get_symbols()


['RRRP3.SA', 'ALOS3.SA', 'ALPA4.SA', 'ABEV3.SA', 'ARZZ3.SA', 'ASAI3.SA', 'AZUL4.SA', 'B3SA3.SA', 'BBSE3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BRAP4.SA', 'BBAS3.SA', 'BRKM5.SA', 'BRFS3.SA', 'BPAC11.SA', 'CRFB3.SA', 'CCRO3.SA', 'CMIG4.SA', 'CIEL3.SA', 'COGN3.SA', 'CPLE6.SA', 'CSAN3.SA', 'CPFE3.SA', 'CMIN3.SA', 'CVCB3.SA', 'CYRE3.SA', 'DXCO3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENGI11.SA', 'ENEV3.SA', 'EGIE3.SA', 'EQTL3.SA', 'EZTC3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'NTCO3.SA', 'SOMA3.SA', 'HAPV3.SA', 'HYPE3.SA', 'IGTI11.SA', 'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'KLBN11.SA', 'RENT3.SA', 'LREN3.SA', 'LWSA3.SA', 'MGLU3.SA', 'MRFG3.SA', 'BEEF3.SA', 'MRVE3.SA', 'MULT3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'RECV3.SA', 'PRIO3.SA', 'PETZ3.SA', 'RADL3.SA', 'RAIZ4.SA', 'RDOR3.SA', 'RAIL3.SA', 'SBSP3.SA', 'SANB11.SA', 'SMTO3.SA', 'CSNA3.SA', 'SLCE3.SA', 'SUZB3.SA', 'TAEE11.SA', 'VIVT3.SA', 'TIMS3.SA', 'TOTS3.SA', 'TRPL4.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VAMO3.SA', 'VBBR3.SA

In [98]:

# Aplicar o nest_asyncio para evitar erros de loop de eventos
nest_asyncio.apply()

async def fetch_single_stock_data(symbol, start_date, end_date):
    df = yf.Ticker(symbol).history(start=start_date, end=end_date)['Close']
    await asyncio.sleep(3)  # Simula um atraso para garantir comportamento assíncrono
    df = df.ffill().bfill()  # Preenche NaNs com o primeiro e último valor válido
    return symbol, df

async def fetch_stock_data(symbols, months):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=months*30)
    all_data = {}

    # Criação de tarefas com asyncio.create_task
    tasks = [asyncio.create_task(fetch_single_stock_data(symbol, start_date, end_date)) for symbol in symbols]

    results = []
    with tqdm(total=len(tasks), desc="Fetching stock data") as pbar:
        for coro in asyncio.as_completed(tasks):
            result = await coro
            results.append(result)
            pbar.n = len([t for t in tasks if t.done()])
            pbar.refresh()
            sleep(1)

    for symbol, df in results:
        all_data[symbol] = df

    combined_df = pd.DataFrame(all_data)
    combined_df.index.name = 'Date'
    return combined_df

# Rodar a função assíncrona
df = asyncio.run(fetch_stock_data(symbols, 6))


Fetching stock data: 100%|██████████| 86/86 [02:30<00:00,  1.75s/it]


In [108]:
def save_to_sqlite(df, db_name, table_name):
    # Conectar ao banco de dados SQLite
    conn = sqlite3.connect(db_name)
    
    # Salvar o DataFrame no banco de dados
    df.to_sql(table_name, conn, if_exists='replace', index=True)
    
    # Fechar a conexão com o banco de dados
    conn.close()
    print(f"Data saved to SQLite database '{db_name}' in table '{table_name}'")

In [107]:
save_to_sqlite(df, "db_bolsa", "tb_historico_acoes")

Data saved to SQLite database 'db_bolsa' in table 'tb_historico_acoes'


In [99]:
def calculate_rentability(df):
    rentability_data = [
        (symbol, df[symbol].iloc[0], df[symbol].iloc[-1], (df[symbol].iloc[-1] - df[symbol].iloc[0]) / df[symbol].iloc[0])
        for symbol in df.columns]
    
    rentability_df = pd.DataFrame(rentability_data, columns=['Empresa', 'Initial Value', 'Final Value', 'Rentability'])
    rentability_df.sort_values('Rentability', ascending=False, inplace=True)
    top_stocks = rentability_df.head(10)
    
    return top_stocks
    
df_top10= calculate_rentability(df)
df_top10


,Empresa,Initial Value,Final Value,Rentability
30,EMBR3.SA,23.799999,36.910000,0.550840
18,CMIG4.SA,7.489175,10.440000,0.394012
19,CIEL3.SA,4.222324,5.650000,0.338126
14,BRFS3.SA,13.820000,18.440001,0.334298
47,JBSS3.SA,23.760000,29.170000,0.227694
59,PETR4.SA,32.032825,38.380001,0.198146
58,PETR3.SA,34.193348,39.939999,0.168063
53,MRFG3.SA,9.550000,11.000000,0.151832
67,SBSP3.SA,67.747192,77.320000,0.141302
84,WEGE3.SA,34.462029,37.810001,0.097150
